In [ ]:
import mediapipe as mp 
import cv2
import numpy as np 
import os
import numpy as np
from pathlib import Path
import xml.etree.cElementTree as ET
from PIL import Image
import time
import gc

In [ ]:
# Gernarates a rectangular box around the hand with the help of landmarks generated by Mediapipe 
# Saves the coordinates of the box generated into a list
def get_rectangle_coordinates(image_path):
    xmincol = []
    ymincol = []
    xmaxcol = []
    ymaxcol = []
    image_name = image_path

    mphands = mp.solutions.hands
    hands = mphands.Hands()
    mp_drawing = mp.solutions.drawing_utils
    cap = cv2.imread(image_name)

    h, w, c = cap.shape

    while True:
        framergb = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            for handLMs in hand_landmarks:
                x_max = 0
                y_max = 0
                x_min = w
                y_min = h
                for lm in handLMs.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    if x > x_max:
                        x_max = x
                    if x < x_min:
                        x_min = x
                    if y > y_max:
                        y_max = y
                    if y < y_min:
                        y_min = y
                xmincol.append(x_min) 
                ymincol.append(y_min)
                xmaxcol.append(x_max)
                ymaxcol.append(y_max)
        try:
            if min(xmincol) <= 10:
                    x_min = 0
            else:
                x_min = min(xmincol) - 10         
            if min(ymincol) <= 10:
                y_min = 0
            else:
                y_min = min(ymincol) - 10
            if max(xmaxcol) >= w:
                x_max = w
            else:
                x_max = max(xmaxcol) + 10
            if max(ymaxcol) >= h:
                y_max = h
            else:
                y_max = max(ymaxcol) + 10
            coord = [x_min, y_min, x_max, y_max]
            del xmaxcol[:]
            del ymaxcol[:]
            del xmincol[:]
            del ymincol[:]
            gc.collect()
            return coord
        except:
            os.remove(image_path)
            break

In [ ]:
# Converts collected box coordinates into an XML file format similar to the format created by LabelImg
# The XML file along with the image combine to form a tensorflow record 
# This tells tensorflow exactly what to focus on for training
def create_labimg_xml(image_path, annotation_list):
    try:
        new_image_path = Path(image_path)
        img = np.array(Image.open(new_image_path).convert('RGB'))

        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'folder').text = str(new_image_path.parent.name)
        ET.SubElement(annotation, 'filename').text = str(new_image_path.name)
        ET.SubElement(annotation, 'path').text = str(new_image_path)

        source = ET.SubElement(annotation, 'source')
        ET.SubElement(source, 'database').text = 'Unknown'

        size = ET.SubElement(annotation, 'size')
        ET.SubElement(size, 'width').text = str (img.shape[1])
        ET.SubElement(size, 'height').text = str(img.shape[0])
        ET.SubElement(size, 'depth').text = str(img.shape[2])

        ET.SubElement(annotation, 'segmented').text = '0'

        for annot in annotation_list:
            tmp_annot = annot.split(',')
            cords, label = tmp_annot[0:4], tmp_annot[-1]
            xmin, ymin, xmax, ymax = cords[0], cords[1], cords[2], cords[3]

            object = ET.SubElement(annotation, 'object')
            ET.SubElement(object, 'name').text = label
            ET.SubElement(object, 'pose').text = 'Unspecified'
            ET.SubElement(object, 'truncated').text = '0'
            ET.SubElement(object, 'difficult').text = '0'

            bndbox = ET.SubElement(object, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = str(xmin)
            ET.SubElement(bndbox, 'ymin').text = str(ymin)
            ET.SubElement(bndbox, 'xmax').text = str(xmax)
            ET.SubElement(bndbox, 'ymax').text = str(ymax)

        tree = ET.ElementTree(annotation)

        save_path = Path(r'SET_SAVE_PATH' %label)
        # Ideally save_path should match image path
        xml_file_name = save_path / (new_image_path.name.split('.')[0]+'.xml')
        tree.write(xml_file_name)
        del annotation_list[:]
        del tree
        del source
        del annotation
        img = None
        del img
        gc.collect()
    except:
        return        

In [ ]:
folder_path = r'SET_FOLDER_PATH'
folders = os.listdir(folder_path)

for folder in folders:
    folder_path = r'SET_FOLDER_PATH'
    folder_path = os.path.join(folder_path, folder)
    label = folder
    print("Folder Path: ", folder_path)
    print("Label:", label)
    
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path,image)
        print("At: ", image_path)
        annotation_list = []
        annotation_list = get_rectangle_coordinates(image_path)
        annotation_list = str(annotation_list)
        annotation_list = annotation_list + "," + label
        annotation_list = annotation_list.replace("[", "")
        annotation_list = annotation_list.replace("]", "")
        annotation_list = annotation_list.replace(" ", "")
        annotation_list = annotation_list.split()
        create_labimg_xml(image_path, annotation_list)
        
        del annotation_list[:]
        gc.collect()
        image_path = ''  